In [1]:
import tensorflow as tf
from __future__ import print_function
import numpy as np

/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
class BatchSampler(object):
    '''
    A (very) simple wrapper to randomly sample batches without replacement.
    '''
    
    def __init__(self, data, targets, batch_size):
        self.num_points = data.shape[0]
        self.features = data.shape[1]
        self.data = data
        self.targets = targets
        self.batch_size = batch_size
        self.indices = np.arange(self.num_points)

    def random_batch_indices(self, m=None):
        if m is None:
            indices = np.random.choice(self.indices, self.batch_size, replace=False)
        else:
            indices = np.random.choice(self.indices, m, replace=False)
        return indices 

    def get_batch(self, m=None):
        '''
        Get a random batch without replacement from the dataset.
        If m is given the batch will be of size m. 
        Otherwise will default to the class initialized value.
        '''
        indices = self.random_batch_indices(m)
        X_batch = np.take(self.data, indices, 0)
        y_batch = self.targets[indices]
        return X_batch, y_batch  


In [3]:
def loadBinData():
# import binary NOTMIST data set
    with np.load("notMNIST.npz") as data :
        Data, Target = data ["images"], data["labels"]
        posClass = 2
        negClass = 9
        dataIndx = (Target==posClass) + (Target==negClass)
        Data = Data[dataIndx]/255.
        Target = Target[dataIndx].reshape(-1, 1)
        Target[Target==posClass] = 1
        Target[Target==negClass] = 0
        np.random.seed(521)
        randIndx = np.arange(len(Data))
        np.random.shuffle(randIndx)
        Data, Target = Data[randIndx], Target[randIndx]
        trainData, trainTarget = Data[:3500], Target[:3500]
        validData, validTarget = Data[3500:3600], Target[3500:3600]
        testData, testTarget = Data[3600:], Target[3600:]
        
    print("Data binary class Loaded")
    print("-------------------------------")
    return trainData, trainTarget,validData, validTarget,\
        testData, testTarget

def arrFlatten(arr):
    
    print(arr.shape)

    dataDim1, dum1, dum2 = arr.shape
    arr = np.reshape(arr, [ dataDim1 ,784 ])
    return arr



def linearMSE(y_hat, target):
    '''
    TODO: the MSE calculation
    '''
    print("y_hat",  y_hat)
    target = tf.cast(target, dtype = tf.float32)
    print("target", target)
    mse_mat = tf.square(tf.subtract(y_hat, target))
    print("msemst", mse_mat)
    loss = tf.reduce_mean(mse_mat)/2.0
    return loss

def linearBuildGraph(_regLambda, _learningRate):
    '''
    Input: _data is x in the equation, dim by 784 flattened tensor
       _target is y in the equaion
       _regLambda is the wegithed decay coeff
       _learningRate is the epsilon
    '''

    #declare using a placeholder, feed in _data and _target to x ,y 
    #x_dim, dum1 =_data.get_shape().as_list()
    X = tf.placeholder(tf.float32, shape=[None, 784], name='dataX')
    # W initialize to a gaussian distr, honestly anything would work
    W = tf.Variable(tf.truncated_normal(shape=[784, 1], stddev=0.1), name='weights')
    W = tf.cast(W, dtype=tf.float32)
    b = tf.Variable(0.0, name='biases')
    y_target = tf.placeholder(tf.float32, shape=[None, 1], name='targetY')
    
    #compute the current y_hat
    y_hat =  tf.matmul(X, W) + b
    #compute the current loss
    mseCurr = linearMSE(y_hat, y_target)
    #compute the decay/regularization term
    regTerm = tf.multiply(_regLambda, tf.reduce_mean(tf.square(W)))
    mseError = tf.add(mseCurr, regTerm)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate = _learningRate)
    train = optimizer.minimize(loss=mseCurr)

    return W, b, mseError, y_hat, X, y_target
    

def runLinearGraph(trainData, trainTarget ):
    
    '''
    Input: _data,
           _target,
           _numIters
    Output:
           required accuracy/epoch plots
           
    '''
    
    regLambda = 0.0
    learningRate = [0.005, 0.001, 0.0001]
    numIter = 20000
    numEpoch =int(np.ceil(20000/7))
    batchSize = 500
    
    tf.reset_default_graph()
    W, b, mseError, y_hat, X, y_target = linearBuildGraph(regLambda, learningRate)
    

    init = tf.global_variables_initializer()
    sess = tf.InteractiveSession()
    sess.run(init)
    initialW = sess.run(W)  
    initialb = sess.run(b)            
    #training model and iter through batches
    #trainBatchSampler = BatchSampler(trainData, trainTarget, batchSize)
    #dataBatch, targetBatch = trainBatchSampler.get_batch()
    currentW, currentb, errTrain, y_hat = sess.run([W, b, mseError, y_hat], feed_dict={X: trainData[0:9], y_target: trainTarget[0:9]})
   
    summary_writer = tf.train.SummaryWriter('some_logdir', linearBuildGraph(regLambda, learningRate))

    
    print(errTrain.eval())
    print("done")
    

if __name__ == '__main__':  
        

    #trainTarget = arrFlatten(trainTarget)
    trainData, trainTarget, validData, validTarget,testData, testTarget = loadBinData()
    #data is (3500, 28, 28)
    #the label [1] or [0] is stored in target
    trainData = arrFlatten(trainData)
    runLinearGraph(trainData, trainTarget)
    

Data binary class Loaded
-------------------------------
(3500, 28, 28)
y_hat Tensor("add:0", shape=(?, 1), dtype=float32)
target Tensor("targetY:0", shape=(?, 1), dtype=float32)
msemst Tensor("Square:0", shape=(?, 1), dtype=float32)


ValueError: Shape must be rank 0 but is rank 1 for 'GradientDescent/update_weights/ApplyGradientDescent' (op: 'ApplyGradientDescent') with input shapes: [784,1], [3], [784,1].